<a href="https://colab.research.google.com/github/akilasachintha/nlp-module-fyp/blob/main/nlp_model_fyp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Mount Google Drive to Google Colab

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


Install Required Packages

In [2]:
!pip install scikit-learn nltk

Import and Download Required Packages

In [3]:
import nltk
import pandas as pd
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import string
import re

nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

Get CSV from Google Drive

In [32]:
data = pd.read_csv('/content/drive/My Drive/Final Year Project/comments.csv')
df = pd.DataFrame(data)
df.head(5)

,_id,parent_id,date,user_id,content
0,1281633928564073_1281713078556158,165836530143824_1281633928564073,2016-11-16T15:07:59+0000,165836530143824,"Hi Matt, thanks for getting in touch. We're so..."
1,1281619061898893_1281712515222881,165836530143824_1281619061898893,2016-11-16T15:06:47+0000,165836530143824,"Hi there, thanks for getting in touch. Yes, ou..."
2,1281563578571108_1281708435223289,165836530143824_1281563578571108,2016-11-16T15:02:37+0000,165836530143824,Hi Sasha and thanks for your comment. We ask t...
3,1281563578571108_1281808185213314,165836530143824_1281563578571108,2016-11-16T17:39:06+0000,10154647591551275,No it does not help what so ever . At no stage...
4,1281563578571108_1282352388492227,165836530143824_1281563578571108,2016-11-17T07:02:29+0000,10209693383358556,Well said Sasha!! It's nobodies fault to grow ...


Comments Preprocessing

* Convert to Lowercase
* Remove Punctuation
* Tokenization
* Remove Stopwords
* Remove Emojies

In [34]:
def remove_emojis(text):
    emoji_pattern = re.compile("["
                           u"\U0001F600-\U0001F64F"  # emoticons
                           u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                           u"\U0001F680-\U0001F6FF"  # transport & map symbols
                           u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           u"\U00002702-\U000027B0"
                           u"\U000024C2-\U0001F251"
                           "]+", flags=re.UNICODE)
    return emoji_pattern.sub(r'', text)

def preprocess_text(text):
    if not isinstance(text, str):
        text = str(text)

    text = remove_emojis(text)
    text = text.lower()
    punctuation = string.punctuation + "’'"
    text = text.translate(str.maketrans('', '', punctuation))
    tokens = word_tokenize(text)
    stop_words = set(stopwords.words('english'))
    filtered_tokens = [word for word in tokens if word not in stop_words]
    cleaned_text = ' '.join(filtered_tokens)
    return cleaned_text if cleaned_text else 'emptycomment'

df['cleaned_comments'] = df['content'].apply(preprocess_text)

Cosine Similarity Based Method Initial Model

In [35]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

def cosine_similarity_between_sentences(sentence1, sentence2):
    tokens = [sentence1, sentence2]
    tfidf_vectorizer = TfidfVectorizer()
    tfidf_matrix = tfidf_vectorizer.fit_transform(tokens)
    cosine_sim = cosine_similarity(tfidf_matrix, tfidf_matrix)
    similarity_score = cosine_sim[0, 1]
    return similarity_score

sentence1 = "This is a sample sentence."
sentence2 = "this is a sample sentence."

similarity_score = cosine_similarity_between_sentences(sentence1, sentence2)
print(f"Cosine Similarity: {similarity_score}")

Cosine Similarity: 1.0


This function returns avetage cosine similarity

In [36]:
def avg_cos_similarity(array1 , array2):
  total_similarity = 0
  x = len(array1)
  y = len(array2)
  similarities = 0
  for i in range(0,x):
    for j in range(0,y):
      print(cosine_similarity_between_sentences(array1[i],array2[j]))
      total_similarity+= cosine_similarity_between_sentences(array1[i],array2[j])
      similarities+= 1
  avg_similarity = total_similarity / similarities
  return avg_similarity

In [37]:
list_of_lists = [[1, 2], [3, 4], [5, 6]]

combined_list = [inner_list[0] + inner_list[1] for inner_list in list_of_lists]

print(combined_list)

[3, 7, 11]


In [38]:
def combine_max_cosine_sets(lists):
  x = len(lists)
  print(x)
  max_cos_similarity = 0
  max_i = 0
  max_j = 0
  for i in range(0,x-1):
    for j in range(i+1,x):
      avg_similarity = avg_cos_similarity(lists[i],lists[j])
      print(avg_similarity)
      if avg_similarity >= max_cos_similarity:
        max_i = i
        max_j = j
        max_cos_similarity = avg_similarity
  print(max_i,max_j)
  new_list = lists[max_i] + lists[max_j]
  print(new_list)
  del lists[max_i]
  del lists[max_j-1]
  lists.append(new_list)
  return lists

In [24]:
def hac_clusters(clusters , lists):
  while len(lists)> clusters:
    lists = combine_max_cosine_sets(lists)
  return lists

In [39]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import string

sentences = [["Your first sentence here."], ["Your first sentence here", "Your second sentence here."], ["Your first sentence here"]]
lists = hac_clusters(1 , sentences)
print(lists)

3
1.0
0.6029748160380572
0.8014874080190286
1.0
1.0
1.0
0.6029748160380572
0.8014874080190286
0 2
['Your first sentence here.', 'Your first sentence here']
2
1.0
1.0
0.6029748160380572
0.6029748160380572
0.8014874080190286
0 1
['Your first sentence here', 'Your second sentence here.', 'Your first sentence here.', 'Your first sentence here']
[['Your first sentence here', 'Your second sentence here.', 'Your first sentence here.', 'Your first sentence here']]


In [40]:
df.head(5)

,_id,parent_id,date,user_id,content,cleaned_comments
0,1281633928564073_1281713078556158,165836530143824_1281633928564073,2016-11-16T15:07:59+0000,165836530143824,"Hi Matt, thanks for getting in touch. We're so...",hi matt thanks getting touch sorry hear welcom...
1,1281619061898893_1281712515222881,165836530143824_1281619061898893,2016-11-16T15:06:47+0000,165836530143824,"Hi there, thanks for getting in touch. Yes, ou...",hi thanks getting touch yes prebuilt gingerbre...
2,1281563578571108_1281708435223289,165836530143824_1281563578571108,2016-11-16T15:02:37+0000,165836530143824,Hi Sasha and thanks for your comment. We ask t...,hi sasha thanks comment ask customers put good...
3,1281563578571108_1281808185213314,165836530143824_1281563578571108,2016-11-16T17:39:06+0000,10154647591551275,No it does not help what so ever . At no stage...,help ever stage cashier explain elderly gentle...
4,1281563578571108_1282352388492227,165836530143824_1281563578571108,2016-11-17T07:02:29+0000,10209693383358556,Well said Sasha!! It's nobodies fault to grow ...,well said sasha nobodies fault grow old privil...


In [41]:
df2 = df.head(1000)

In [42]:
sentences = df2['cleaned_comments']
lists_of_sentences = [[sentence] for sentence in sentences]
list_of_lists = [sentence_list for sentence_list in lists_of_sentences]
print("\nList of Lists:")
for sentence_list in list_of_lists:
    print(sentence_list)


List of Lists:
['hi matt thanks getting touch sorry hear welcome return chicken bites store along proof purchase full refund meantime well sure pass feedback relevant team many thanks aldi uk facebook team']
['hi thanks getting touch yes prebuilt gingerbread house stores many thanks aldi uk facebook team']
['hi sasha thanks comment ask customers put goods back trolley scanned checkout customers pack leisure long shelf behind tills customers using basket ask place basket start conveyor belt pack items directly carrier bags scanned hope helps clear confusion thanks commenting aldi uk facebook team']
['help ever stage cashier explain elderly gentleman store state anything written suppose mind readers inform elderly people neighbourhood shop elsewhere future aldi clearly think mind readers clearly time patience elderly']
['well said sasha nobodies fault grow old privilege often denied many aldi dont packers available everyone able move quickly convenience shall advise friends carers elder

In [ ]:
lists = hac_clusters(5 , list_of_lists)
print(lists)

Streaming output truncated to the last 5000 lines.
0.0
0.0
0.02812575785353808
0.02812575785353808
0.0
0.0
0.03469236704547868
0.03469236704547868
0.07433426458775619
0.07433426458775619
0.31226151500403715
0.31226151500403715
0.0
0.0
0.3646822325029499
0.3646822325029499
0.3861662434292926
0.3861662434292926
0.31340871333227743
0.31340871333227743
0.3762029150570235
0.3762029150570235
0.41117728067149534
0.41117728067149534
0.1762588503849108
0.1762588503849108
0.37139810744039903
0.37139810744039903
0.3508270977096491
0.3508270977096491
0.0
0.0
0.042952592697600114
0.042952592697600114
0.0
0.0
0.25334404472090116
0.25334404472090116
0.53598168531369
0.53598168531369
0.0
0.0
0.04704829436004893
0.04704829436004893
0.0
0.0
0.0
0.0
0.32762304653776547
0.32762304653776547
0.32762304653776547
0.32762304653776547
0.0
0.0
0.0
0.0
0.0
0.0
0.3646822325029499
0.3646822325029499
0.0
0.0
0.1986441281224999
0.1986441281224999
0.3541590184333038
0.3541590184333038
0.05961747793358326
0.05961747793

In [ ]:
print(len(lists))

1
